# IZV Úkol 2: Test hypotézy

Import potřebných knihoven pro zpracování statistických dat. Knihovna `scipy.stats` obsahuje funkce pro testování hypotéz.


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency


Načtení datového rámcu


In [2]:
df = pd.read_pickle("accidents.pkl.gz")
df


,p1,p36,p37,p2a,weekday(p2a),p2b,p6,p7,p8,p9,...,l,n,o,p,q,r,s,t,p5a,region
0,002100160001,4,-1,2016-01-01,5,55,1,1,0,2,...,,711403,,Souhlasnýsesměremúseku,Pomalý,554782,451622,GN_V0.1UIR-ADR_410,1,PHA
1,002100160002,4,-1,2016-01-01,5,130,1,3,0,2,...,,,,,,-1,-1,,1,PHA
2,002100160003,5,-1,2016-01-01,5,100,1,2,0,2,...,,,,,,-1,-1,,1,PHA
3,002100160004,6,-1,2016-01-01,5,120,9,0,0,2,...,,,,,,-1,-1,,1,PHA
4,002100160005,6,-1,2016-01-01,5,2560,2,0,0,2,...,,,,,,-1,-1,,1,PHA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572929,190906210457,6,-1,2021-08-30,1,506,2,0,0,2,...,,2611387,,,Pomalý,560359,-1,GN_V0.1UIR-ADR_410,1,KVK
572930,190906210458,6,-1,2021-08-30,1,752,2,0,0,2,...,,2541503,,Opačnýkesměruúseku,Pomalý,560383,550426,GN_V0.1UIR-ADR_410,1,KVK
572931,190906210459,3,2099,2021-08-30,1,1505,1,3,0,2,...,,,,,,-1,-1,,1,KVK
572932,190906210460,3,2099,2021-08-31,2,1245,0,0,0,1,...,2099,199764,,Opačnýkesměruúseku,Pomalý,560286,553654,GN_V0.1UIR-ADR_410,1,KVK


## Hypotéza 1:

**Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích třetí třídy.**

Hypotézu budeme ověřovat s 95% jistotou.

Přidáme pomocný sloupec `death`, který obsahuje informaci o tom, zda při nehodě došlo k umrtí jedné nebo více osob.
Z datového rámce vybereme data pro nehody na silnicích první a třetí třídy.


In [3]:
df["death"] = df["p13a"] > 0
df_sel = df[df["p36"].isin((1, 3))]


Kontigenční tabulka obsahující počet nehod podle
- třídy silnice - `1` / `3` 
- toho, zda došlo k umrtí nebo ne - `True` / `False`

In [4]:
ct = pd.crosstab(df_sel["death"], df_sel["p36"])
ct


p36,1,3
death,,
False,78618,73352
True,911,448


### $\chi^2$ test

Pro výpočet $\chi^2$ testu použijeme funkci `chi2_contingency`. Výstupem funkce jsou hodnoty (chi2, p-value, dof, expected).


In [5]:
res = chi2_contingency(ct)
res

(125.72070150000258,
 3.5395243450138555e-29,
 1,
 array([[78824.11109444, 73145.88890556],
        [  704.88890556,   654.11109444]]))

**Hypotéza 1 - Závěr**


Hodnota $3.54\cdot10^{-29}$ je mnohem menší než $0.05$. Tím pádem **zamítáme nulovou hypotézu** a prohlašujeme, že **existuje statisticky významná korelace mezi smrtelnými nehodami a silnicemi 1. třídy**.



## Hypotéza 2:

**Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.**

Hypotézu budeme ověřovat s 95% jistotou.

Z datového rámce vybereme data pro vozidla značky Škoda (39) a Audi (2) - sloupec *p45a*.

In [6]:
df_sel = df[df["p45a"].isin((2, 39))]


Kontigenční tabulka obsahující počet nehod podles 
- značky auta
- škody na vozidle (sloupec *p53*)


In [7]:
ct = pd.crosstab(df_sel["p53"], df_sel["p45a"])
ct

p45a,2,39
p53,,
0,764,8203
1,12,240
2,1,74
3,1,13
4,0,7
...,...,...
9000,4,2
10000,10,8
12000,2,0


### T-Test

Pro výpočet t-testu použijeme funkci `ttest_ind`. Výstupem funkce jsou hodnoty `t-statistic` a `p-value`.

In [8]:
ttest_ind(ct[2], ct[39], equal_var=False)

Ttest_indResult(statistic=-3.417269886834147, pvalue=0.0007857942843984687)

**Hypotéza 2 - Závěr**

Hodnota $0.0008$ (`pvalue`) je menší než $0.05$. Proto **zamítáme nulovou hypotézu**.
